In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/nlp_project

/content/drive/My Drive/nlp_project


In [ ]:
%ls Documents

'AFFAIRE A.A.K. c. TÜRKiYE.pdf'


In [ ]:
import os
os.getcwd()

'/content/drive/My Drive/nlp_project'

In [ ]:
!ls

Documents  output.pdf  requirements.txt


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Pinecone
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [ ]:
from langchain.llms import HuggingFaceHub
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("HF token")

HF token··········


In [ ]:
def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents

In [ ]:
directory = 'Documents/'
documents = load_docs(directory)
doc_len = len(documents)
doc_len

23

In [ ]:
documents[22]

Document(page_content='ARRÊT A.A.K. c. TÜRKİYE – OPINION SÉPARÉE\n22eu des raisons solides pour examiner d’office s’il fallait désigner un autre \navocat. Or il n’existe aucune trace d’un tel examen.\n95.  Sixièmement, dès lors que l’on rappelle à nouveau le fait que la \nrequérante n’était pas en mesure, selon le Gouvernement, de prendre des \ndécisions éclairées pour son propre compte, le fait qu’elle n’ait pas demandé \nau tribunal de désigner un nouvel avocat n’est guère pertinent. En outre, bien \nque la requérante ait dénoncé dans son recours l’absence d’accès à un avocat, \nla décision très succincte de la Cour de cassation ne porte aucune appréciation \nsur cette question.\n9.  Malgré les arguments que j’ai exposés ci-dessus, j’ai voté en faveur \nd’un constat de non-violation de l’article 8. En effet, je souscris à l’avis de \nmes collègues selon lequel, eu égard à l’ensemble des faits de l’affaire, \nnotamment à toutes les mesures prises pour sauvegarder les intérêts de la \n

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import torch

# # Load pre-trained XLM-RoBERTa model and tokenizer
# model_name = "xlm-roberta-base"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# # Define the text for language identification
# text = " Respect des biens"

# # Tokenize the text
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# # Forward pass through the model
# outputs = model(**inputs)

# # Get the predicted language
# predicted_language_id = torch.argmax(outputs.logits, dim=1).item()

# # Language codes corresponding to XLM-RoBERTa
# language_codes = ['ar', 'bg', 'bn', 'de', 'en', 'es', 'et', 'fa', 'fi', 'fr', 'gu', 'hi', 'hr', 'id', 'it', 'ja', 'kk', 'kn', 'ko', 'lt', 'lv', 'mk', 'ml', 'mr', 'ms', 'my', 'nl', 'no', 'pl', 'pt', 'ro', 'ru', 'si', 'sk', 'sl', 'sq', 'sv', 'ta', 'te', 'th', 'tl', 'tr', 'uk', 'ur', 'vi', 'zh']

# # Map the predicted language ID to language code
# predicted_language_code = language_codes[predicted_language_id]

# print("Predicted Language:", predicted_language_code)

In [ ]:
from transformers import AutoTokenizer, TFMBartModel, MBartForConditionalGeneration
import tensorflow as tf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

In [ ]:
results = []
for i in range(10):
  text = str(documents[i])
  encoded_docs = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(**encoded_docs)
  translated_docs = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  results.append(translated_docs)

In [ ]:
# text = str(documents[:1])
# encoded_docs = tokenizer(text, return_tensors="pt")
# generated_tokens = model.generate(**encoded_docs)
# translated_docs = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# translated_docs

In [ ]:
results

[["page_content='SECOND SECTION\\nA.A.K. v. TÜRKİYE\\n(Application No. 56578/11)\\n JUDGMENT\\nArticle 8 \\nPrivacy \\nThe applicant has been placed under judicial supervision following a \\nprocedure which concluded that she suffered from a mental disorder which impaired her ability to act \\n Existence of effective safeguards in domestic procedure to prevent abuses by ensuring that the applicant’s rights and interests are taken into account \\nParticipation of the applicant in the decision-making process at all stages of the procedure \\nA judicial mechanism which has proceeded with the necessary fairness and diligence and which has gathered sufficient evidence to assess the applicant’s deficiencies and to prevent possible injustices Limitation of the measure in"],
 ['page_content=\' JUDGMENT A.A.K. v. TÜRKİYE \\n1In the case of A.A.K. v. Türkiye,\\nThe European Court of Human Rights (Second Section), sitting \\na Chamber composed of:\\nArnfinn Bårdsen, President,\\nJovan Ilievski,\\

In [ ]:

for index, text in enumerate(results):
    paragraphs = text[0].split('\\n')
    clean_paragraphs = [para.strip() for para in paragraphs if para.strip()]
    results[index] = clean_paragraphs


In [ ]:
results

[["page_content='SECOND SECTION",
  'A.A.K. v. TÜRKİYE',
  '(Application No. 56578/11)',
  'JUDGMENT',
  'Article 8',
  'Privacy',
  'The applicant has been placed under judicial supervision following a',
  'procedure which concluded that she suffered from a mental disorder which impaired her ability to act',
  'Existence of effective safeguards in domestic procedure to prevent abuses by ensuring that the applicant’s rights and interests are taken into account',
  'Participation of the applicant in the decision-making process at all stages of the procedure',
  'A judicial mechanism which has proceeded with the necessary fairness and diligence and which has gathered sufficient evidence to assess the applicant’s deficiencies and to prevent possible injustices Limitation of the measure in'],
 ["page_content=' JUDGMENT A.A.K. v. TÜRKİYE",
  '1In the case of A.A.K. v. Türkiye,',
  'The European Court of Human Rights (Second Section), sitting',
  'a Chamber composed of:',
  'Arnfinn Bårdsen,

In [ ]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.7 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

In [ ]:
def create_pdf(results):
    doc = SimpleDocTemplate("output.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    content = []

    for result in results:
        text = "<br/>".join(result)
        para = Paragraph(text, style=styles["Normal"])
        content.append(para)
        content.append(Paragraph("<br/><br/>", style=styles["Normal"]))

    doc.build(content)

create_pdf(results)

In [ ]:
def split_docs(documents, chunk_size=512, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                   separators=["\n\n", "\n", " ", ""])
    docs = []
    for doc in documents:
        text = " ".join(doc)
        chunks = text_splitter.split_text(text)
        docs.extend(chunks)
    return docs

In [ ]:
law_results = split_docs(results)
print(len(law_results))

20


In [ ]:
law_results

["page_content='SECOND SECTION A.A.K. v. TÜRKİYE (Application No. 56578/11) JUDGMENT Article 8 Privacy The applicant has been placed under judicial supervision following a procedure which concluded that she suffered from a mental disorder which impaired her ability to act Existence of effective safeguards in domestic procedure to prevent abuses by ensuring that the applicant’s rights and interests are taken into account Participation of the applicant in the decision-making process at all stages of the",
 'all stages of the procedure A judicial mechanism which has proceeded with the necessary fairness and diligence and which has gathered sufficient evidence to assess the applicant’s deficiencies and to prevent possible injustices Limitation of the measure in',
 'page_content=\' JUDGMENT A.A.K. v. TÜRKİYE 1In the case of A.A.K. v. Türkiye, The European Court of Human Rights (Second Section), sitting a Chamber composed of: Arnfinn Bårdsen, President, Jovan Ilievski, Egidijus Kūris, Saadet

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# query_result = embeddings.embed_query("Hello Buddy")
# len(query_result)

In [ ]:
# query_result

In [ ]:
from pinecone import Pinecone as PineconeClient
from langchain_community.vectorstores import Pinecone

os.environ["PINECONE_API_KEY"] = getpass("PINECONE_API_KEY:")

PINECONE_API_KEY:··········


In [ ]:
PineconeClient(api_key="PINECONE_API_KEY", environment="gcp-starter")
index_name = "myindex"
index = Pinecone.from_texts(law_results, embeddings, index_name=index_name)

In [ ]:
def get_similar_docs(query, k=4):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs

In [ ]:
rd = get_similar_docs("On what grounds did the Aydın Bar rejected the applicant’s request for legal assistance?")
rd

[Document(page_content='hospital. The judge ordered the local security directorate to carry out this measure. 16. By decision of 11 June 2009, the Aydın Bar rejected the applicant’s request for legal assistance on the grounds that she had a monthly income of between 600 and 1000 Turkish'),
 Document(page_content="page_content='APPEAL A.A.K. v Turkey 4psychiatry of the Faculty of Forensic Medicine of the Istanbul University Hospital. 24. At the hearing on 17 September 2009, Mr M.A. stated that she had resigned from her duties (paragraph 22 above) and that the applicant needed to be informed. The court read out a faxed memorandum sent by the applicant. 25. By decision of 24 September 2009, the Aydın Bar considered Mr M.A.’s apology to be well founded and annulled the legal aid granted to the applicant. 26. At the next"),
 Document(page_content="page_content='DECISION A.A.K. v Turkey 3qualified to be her legal guardian and to have an inventory of her patrimony. According to the informatio

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
from langchain.llms import HuggingFaceEndpoint
import os
from getpass import getpass

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("HF token")

HF token··········


In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-base",
                   model_kwargs={"temperature":0.1,"max_new_tokens":250})
llm

HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-base', timeout=None)>, repo_id='google/flan-t5-base', task='text2text-generation', model_kwargs={'temperature': 0.1, 'max_new_tokens': 250})

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff", verbose=False)

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    Based only on the summaries provided below, answer the given question.
    If you don't know the answer, respond with "I don't know".
    Do not generate new questions.
    Do not make up your own answer from outside the source material.
    Do not write anything after the answer.

    Source material: {context}

    Question: {question}
    Answer:
    """
)

chain = load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
)

In [ ]:
def get_answer(query):
  relevant_docs = get_similar_docs(query)
  response = chain.run(input_documents=relevant_docs, question=query)
  return response

In [ ]:
our_query = "On what grounds did the Aydın Bar rejected the applicant’s request for legal assistance?"
answer = get_answer(our_query)
print(answer)

She had a monthly income of between 600 and 1000.


In [ ]:
our_query = "Who is supreme court judge?"
answer = get_answer(our_query)
print(answer)

I don't know.
